# Amazon Bedrock Agent

This notebook covers steps to create Amazon Bedrock Agent. Agents for Amazon Bedrock offers you the ability to build and configure autonomous agents in your application. The agent helps your end-users complete actions based on organization data and user input. Agents orchestrate interactions between foundation models, data sources, software applications, and user conversations, and automatically call APIs to take actions and invoke knowledge bases to supplement information for these actions. Developers can easily integrate the agents and accelerate delivery of generative AI applications saving weeks of development effort.

In this notebook, we will create Amazon Bedrock knowledge base, create Lambda functions that perform actions, combine them as action groups, add schema for the actions and integrate with Knowledge base created in a previous step. 

To run this notebook, assumed role needs to have permissions to 
* Create IAM role and policies
* Access Bedrock
* Create Bedrock Agents
* Create Lambda functions
* Upload to S3

This notebook is a fork of Bedrock ImmersionDay notebook here https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/07_Agents/insurance_claims_agent/with_kb/create_and_invoke_agent_with_kb.ipynb. 

## Pre-requisites & Install dependencies

In [ ]:
#Check Python version is greater than 3.8 which is required by Langchain if you want to use Langchain
import sys
sys.version

In [ ]:
!pip install -U boto3
!pip install -U botocore
!pip install -U awscli

## Restart Kernel

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

## Import dependencies

In [1]:
import sys
assert sys.version_info >= (3, 8)

In [2]:
import sagemaker
import boto3
import json
import random
import time
import zipfile
from io import BytesIO
import uuid
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
iam = boto3.client('iam')
s3 = boto3.client('s3')
lambda_c = boto3.client('lambda')
sts = boto3.client('sts')

In [4]:
session = boto3.Session()
sagemaker_session = sagemaker.Session()
studio_region = sagemaker_session.boto_region_name 
caller_identity = sagemaker_session.get_caller_identity_arn()
account_id = sts.get_caller_identity()["Account"]

In [5]:
suffix = f"{studio_region}-{account_id}"
default_bucket = sagemaker_session.default_bucket()
default_bucket_arn = f"arn:aws:s3:::{default_bucket}"

agent_name = "insurance-claims-agent-kb"
agent_alias_name = "workshop-alias"
bedrock_agent_bedrock_allow_policy_name = f"ica-bedrock-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"ica-s3-allow-{suffix}"
bedrock_agent_kb_allow_policy_name = f"ica-kb-allow-{suffix}"
lambda_role_name = f'{agent_name}-lambda-role-{suffix}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_ica'
lambda_file_name = "lambda_function.py"
lambda_code_path = f"agent_assets/{lambda_file_name}"
lambda_name = f'{agent_name}-{suffix}'

schema_key = f'{agent_name}-schema.json'
schema_name = 'insurance_claims_agent_openapi_schema_with_kb.json'
schema_arn = f'arn:aws:s3:::{default_bucket}/{schema_key}'

## Create IAM policies and Role for Bedrock Agent

In this step, we will create the agent policies that allow bedrock model invocation and s3 bucket access.

In [6]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{studio_region}::foundation-model/anthropic.claude-v2:1"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)


In [7]:
bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

In [8]:
bedrock_agent = boto3.client('bedrock-agent')
kb_name = f'insurance-claims-kb-{suffix}'
kb_response = bedrock_agent.list_knowledge_bases(maxResults=30)

In [ ]:
knowledge_base_id = ""

for kb in kb_response["knowledgeBaseSummaries"]:
    if kb["name"] == kb_name:
        knowledge_base_id = kb["knowledgeBaseId"]
        break

assert knowledge_base_id != "", "Knowledge base has not been created. Check before proceeding" 
print(f"Knowledge base ID is {knowledge_base_id}")
knowledge_base_arn = bedrock_agent.get_knowledge_base(knowledgeBaseId=knowledge_base_id)["knowledgeBase"]["knowledgeBaseArn"]
print(f"Knowledge ARN is {knowledge_base_arn}")

In [10]:
bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)
agent_kb_schema_policy = iam.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)

In [11]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

iam.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'bf0b7b8b-33e9-4bd8-bc65-7d6608195cc3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bf0b7b8b-33e9-4bd8-bc65-7d6608195cc3',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Fri, 19 Jan 2024 15:36:24 GMT'},
  'RetryAttempts': 0}}

## Creating Agent
In this step, we will create the Bedrock Agent. We will use create_Agent API to do this and will associate knowledge base with it. Agent supports following workflow steps
* Pre processing
* Orchestration
* Knowledge base response generation
* Post processing

Here we are utilizing the default templates for pre-processing, orchestration, knowledge base response generation and post processing.  You can override the default templates. You can also pass a name of a Lambda function to parse the raw foundation model output in parts of the agent sequence. If you specify this field, at least one of the promptConfigurations must contain a parserMode value that is set to OVERRIDDEN.

For details see the API documentation page https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent_CreateAgent.html

In [12]:
# Create Agent
agent_instruction = """
You are an agent that can handle various tasks related to insurance claims, including looking up claim 
details, finding what paperwork is outstanding, and sending reminders. Only send reminders if you have been 
explicitly requested to do so. If an user asks about your functionality, provide guidance in natural language 
and do not include function names on the output."""

response = bedrock_agent.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent for handling insurance claims.",
    idleSessionTTLInSeconds=1800,
    foundationModel="anthropic.claude-v2:1",
    instruction=agent_instruction,
)

In [ ]:
response

In [ ]:
agent_id = response['agent']['agentId']
agent_id

## Create Lambda function & upload schema
In this step we will create a Lambda function that will be part of the action group. This will be invoked by the agent and Lambda functions perform domain specific functions and actions. Agents will be configured with the Lambda functions and the interface to the function via schema. We will upload the schema, create an IAM role for Lambda function and create the Lambda function

In [15]:
# Upload Open API schema to this s3 bucket
s3.upload_file("agent_assets/" + schema_name, default_bucket, schema_key)

In [16]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam.get_role(RoleName=lambda_role_name)

iam.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': 'b29ff650-ab2e-4c51-8d58-14391666170e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b29ff650-ab2e-4c51-8d58-14391666170e',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Fri, 19 Jan 2024 15:36:39 GMT'},
  'RetryAttempts': 0}}

In [17]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path,lambda_file_name)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_c.create_function(
    FunctionName=lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Create Agent Action Group

We will now create and agent action group that uses the lambda function and API schema created in previous step. The create_agent_action_group API wil be used to create action group. We will use DRAFT as the agent version since we haven't created an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [18]:
# Pause to make sure agent is created. From the below call you will be able to see the Agent status, instruction which we provided in the previous step. 
#You can also view the pre-processing, orchestration, knowledge base response generation and post processing templates
#Agent status will be one of the avalues 'CREATING'|'PREPARING'|'PREPARED'|'NOT_PREPARED'|'DELETING'|'
print(f"Agent status {bedrock_agent.get_agent(agentId=agent_id)['agent']['agentStatus']}")
bedrock_agent.get_agent(agentId=agent_id)["agent"]["promptOverrideConfiguration"]

Agent status NOT_PREPARED


{'promptConfigurations': [{'promptType': 'KNOWLEDGE_BASE_RESPONSE_GENERATION',
   'promptCreationMode': 'DEFAULT',
   'promptState': 'ENABLED',
   'basePromptTemplate': "You are a question answering agent. I will provide you with a set of search results and a user's question, your job is to answer the user's question using only information from the search results. If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question. Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.\n\nHere are the documents for you to reference:\n<documents>\n$search_results$\n</documents>\n\nIf you reference information from a search result within your answer, you must include a citation to source where the information was found. Each result has a corresponding source ID that you should reference. Please output your answer in the follo

In [19]:

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='ClaimManagementActionGroup',
    apiSchema={
        's3': {
            's3BucketName': default_bucket,
            's3ObjectKey': schema_key
        }
    },
    description='Actions for listing claims, identifying missing paperwork, sending reminders'
)

In [ ]:
agent_action_group_response

In [21]:
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']
print(f"Agent action Group id {action_group_id} and Group name {action_group_name}")

Agent action Group id 9DCXEQW8MP and Group name ClaimManagementActionGroup


## Configure Agent to invoke Action Group Lambda

Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created


In [22]:
# Create allow invoke permission on lambda
response = lambda_c.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{studio_region}:{account_id}:agent/{agent_id}",
)

## Associate Bedrock agent with the knowledge base

In [23]:
agent_kb_description = bedrock_agent.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to detail the requirements of each missing document in a insurance claim.',
    knowledgeBaseId=knowledge_base_id 
)

## Prepare DRAFT version of the agent & create agent alias

An alias is a pointer to a specific version that will allow you to test and use that version of an agent within an application. Provide the information below to the alias.

In [ ]:
agent_prepare = bedrock_agent.prepare_agent(agentId=agent_id)
agent_prepare

In [25]:
#Check if the agent status is prepared
agent_status = bedrock_agent.get_agent(agentId=agent_id)['agent']['agentStatus']
print(f'Agent status is {agent_status}')

Agent status is PREPARED


In [26]:
assert agent_status == "PREPARED", "Agent is not prepared yet"
agent_alias = bedrock_agent.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

In [ ]:
agent_alias_id = agent_alias["agentAlias"]["agentAliasId"]
print(f"Agent Alias Id {agent_alias_id}")

In [28]:
agent_alias_status = bedrock_agent.get_agent_alias(agentId=agent_id,agentAliasId=agent_alias_id)["agentAlias"]["agentAliasStatus"]
print(f'Agent alias status is {agent_alias_status}')

Agent alias status is PREPARED


In [29]:
assert agent_alias_status == "PREPARED", "Agent alias is not prepared yet"

## Invoke the Agent
Now that we've created the agent, let's use the bedrock-agent-runtime client to invoke this agent and perform some tasks.

In [30]:
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')

In [31]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime.invoke_agent(
    inputText="send reminder to claim-006. Include the missing documents and their requirements",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            print(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            print(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

## Agent version and alias

Version is a snapshot that preserves the associated objects/ artifacts created with it. A given version has following information associated
* Agent instruction
* Time stamp - created, updated
* Failure reasons & recommendations to resolve the failures
* Prompt information for types 'PRE_PROCESSING','ORCHESTRATION','POST_PROCESSING','KNOWLEDGE_BASE_RESPONSE_GENERATION'
  - Is it default or overridden
  - Prompt template text
  - Inference config- temperature, TopP, TopK, max length, stop sequence
  - Output parser - default or overridden and Lambda function if it is overridden


To depoly an agent and integrate with the application, we need to create an alias. Alias is a pointer to a version of the agent. We can either use an existing version or create a new version when we create an alias. We can optionally associate an alias with a different version. Due to this, you can efficiently switch between different versions of your agent without requiring the application to keep track of the version. For example, you can change an alias to point to a previous version of your agent if there are changes that you need to quickly revert.

NOTE: The working draft version is DRAFT and the alias that points to it is the TestAlias.

In [ ]:
#Get agent versions; You would see DRAFT version and other versions
list_agent_version_response = bedrock_agent.list_agent_versions(agentId=agent_id)
list_agent_version_response

In [ ]:
#Get details of an agent version
version_id = list_agent_version_response["agentVersionSummaries"][0]["agentVersion"]
get_agent_version_response  = bedrock_agent.get_agent_version(agentId=agent_id,agentVersion=version_id)
get_agent_version_response

In [ ]:
#List agent aliases
list_agent_aliases_response = bedrock_agent.list_agent_aliases(agentId=agent_id)
list_agent_aliases_response

In [ ]:
#Get details of an agent alias
alias_id = list_agent_aliases_response["agentAliasSummaries"][0]["agentAliasId"]
get_agent_alias_response  = bedrock_agent.get_agent_alias(agentId=agent_id,agentAliasId=alias_id)
get_agent_alias_response

## Clean-up (Optonal)

We have created multiple components in this notebook and this will perform clean-up and remove them
* update the action group to disable it
* delete agent action group
* delete agent alias
* delete agent
* delete lambda function

In [ ]:
#Set the status of agent group as disabled
update_agent_action_group_response = bedrock_agent.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    apiSchema={
        's3': {
            's3BucketName': default_bucket,
            's3ObjectKey': schema_key
        }
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
) 

In [ ]:
#Delete agent alias
agent_alias_deletion = bedrock_agent.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias_id
)

In [ ]:
#Delete the agent
agent_deletion = bedrock_agent.delete_agent(
    agentId=agent_id
)

In [ ]:
# Delete Lambda function
lambda_c.delete_function(
    FunctionName=lambda_name
)

In [ ]:
# Remove Open API schema saved to s3 bucket
s3.delete_object(Bucket=default_bucket, Key=schema_key)


In [ ]:
# Delete IAM Roles and policies
for policy in [
    agent_bedrock_policy, 
    agent_s3_schema_policy, 
    agent_kb_schema_policy
]:
    response = iam.list_entities_for_policy(
        PolicyArn=policy['Policy']['Arn'],
        EntityFilter='Role'
    )

    for role in response['PolicyRoles']:
        iam.detach_role_policy(
            RoleName=role['RoleName'], 
            PolicyArn=policy['Policy']['Arn']
        )

    iam.delete_policy(
        PolicyArn=policy['Policy']['Arn']
    )

    
iam.detach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [
    agent_role_name, 
    lambda_role_name, 
]:
    try: 
        iam.delete_role(
            RoleName=role_name
        )
    except Exception as e:
        print(e)
        print("couldn't delete role", role_name)
        